In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import datetime

import time
from tqdm.auto import tqdm

import requests
import glob
import os
import shutil


# 1. Weather data: 

Source: [Deutscher Wetter Dienst (German Weather Service)](https://www.dwd.de/DE/Home/home_node.html)

[Python API](https://wetterdienst.readthedocs.io/en/latest/overview.html), 
[Rest API](https://dwd.api.bund.dev/)

More information about the data can be found [here](https://www.dwd.de/DE/Home/home_node.html;jsessionid=5F6860A891AB172540563F2A56F045B3.live31092)


In [62]:
from wetterdienst import Wetterdienst
from wetterdienst.provider.dwd.observation import DwdObservationRequest, DwdObservationDataset, DwdObservationPeriod, DwdObservationResolution

# Create API instance
API = Wetterdienst(provider="dwd", network="observation")

# Create request
request = DwdObservationRequest(
    parameter=[DwdObservationDataset.SOLAR, DwdObservationDataset.WIND],
    resolution=DwdObservationResolution.HOURLY,
    start_date=datetime.datetime(2023, 1, 1),
    end_date=datetime.datetime(2024, 2, 18),
    # period=DwdObservationPeriod.HISTORICAL,
)

# Create df for available stations
d_stations = request.all().df.to_pandas()
d_stations = d_stations[['station_id', 'state', 'name']]

# Parameters that should be fetched
parameters = ['wind_speed', 'sunshine_duration', 'radiation_global']

# Print parameters that are available with hourly resolution
DwdObservationRequest.discover()['hourly']

{'temperature_air_mean_200': {'origin': '°C', 'si': 'K'},
 'humidity': {'origin': 'pct', 'si': 'pct'},
 'cloud_type_layer1': {'origin': '-', 'si': '-'},
 'cloud_height_layer1': {'origin': 'm', 'si': 'm'},
 'cloud_cover_layer1': {'origin': '1/8', 'si': 'pct'},
 'cloud_type_layer2': {'origin': '-', 'si': '-'},
 'cloud_height_layer2': {'origin': 'm', 'si': 'm'},
 'cloud_cover_layer2': {'origin': '1/8', 'si': 'pct'},
 'cloud_type_layer3': {'origin': '-', 'si': '-'},
 'cloud_height_layer3': {'origin': 'm', 'si': 'm'},
 'cloud_cover_layer3': {'origin': '1/8', 'si': 'pct'},
 'cloud_type_layer4': {'origin': '-', 'si': '-'},
 'cloud_height_layer4': {'origin': 'm', 'si': 'm'},
 'cloud_cover_layer4': {'origin': '1/8', 'si': 'pct'},
 'cloud_cover_total': {'origin': '1/8', 'si': 'pct'},
 'cloud_cover_total_index': {'origin': '-', 'si': '-'},
 'temperature_dew_point_mean_200': {'origin': '°C', 'si': 'K'},
 'wind_gust_max': {'origin': 'm / s', 'si': 'm / s'},
 'humidity_absolute': {'origin': '-', 'si

In [17]:
fetch_data = True
if fetch_data:
    # Loop through all stations and add values for all parameters to dictionary
    # Structure of dictionary: key: parameter_state (i.e. wind_speed_Berlin)
    # Note: For each key, values are 2d since there are multiple stations per state
    data = {}
    for idx in tqdm(range(len(d_stations))):
        d = request.filter_by_station_id(
            station_id=(d_stations['station_id'][idx], ))
        state = d_stations['state'][idx]

        # do actual query
        for result in d.values.query():
            d = result.df.to_pandas()
        if idx == 0:
            date = d['date'][d['parameter'] == 'end_of_interval']
            data['time_unix'] = [dt.timestamp() for dt in list(date)]

        # Get values for each parameter
        for param in parameters:
            values = d[d['parameter'] == param]['value']
            if sum(values.isna()) < 0.2*len(values):
                if not state == None:
                    column_name = param+'_'+state
                    if column_name in data:
                        data[column_name].append(list(values))
                    else:
                        data[column_name] = [list(values)]

  0%|          | 0/447 [00:00<?, ?it/s]

In [35]:
df = data.copy()

for key in list(df.keys()):
    # print(key)
    df[key] = np.nanmean(df[key], axis=0)

df = pd.DataFrame(df)
df['time_unix'] = data['time_unix']
df.to_csv('data/Weather_data.csv')
df.head()

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

# 2. Energy data: 

Source: [Energy-charts.info](https://energy-charts.info/)

[Rest API](https://api.energy-charts.info/)
Here, I collect 'public_power', note that this does not include power produced for industrial self supply

The website is maintained by Fraunhofer Institute for Solar Energy Systems (ISE) a German organization for applied research.
ISE collects data from both federal agencies and private companies, see [here](https://energy-charts.info/sources.html?l=en&c=DE
) for more information 

Further information about the different categories of power production can be found [here](https://energy-charts.info/explanations.html?l=en&c=DE)

In [20]:
# Format: YYYY-MM-DD
date_start = '2023-01-01'
date_end = '2024-02-18'

# 'de' for Germany
country_code = 'de'

# Get request
t0 = time.time()
response = requests.get(
    f'https://api.energy-charts.info/public_power?country={country_code}&start={date_start}T00%3A00%2B01%3A00&end={date_end}T23%3A45%2B01%3A00')
print(response)
print(time.time()-t0)

<Response [200]>
8.637905836105347


In [21]:
# Extract data from response and convert to pandas df
data = {}
t = response.json()['unix_seconds']
data.update({'time_unix': t})
for elem in response.json()['production_types']:
    data.update({elem['name']: elem['data']})

df = pd.DataFrame(data)

# Save df as csv
df.to_csv('data/Energy_data.csv')
df.head()

,time_unix,Hydro pumped storage consumption,Cross border electricity trading,Nuclear,Hydro Run-of-River,Biomass,Fossil brown coal / lignite,Fossil hard coal,Fossil oil,Fossil gas,...,Hydro pumped storage,Others,Waste,Wind offshore,Wind onshore,Solar,Load,Residual load,Renewable share of generation,Renewable share of load
0,1672527600,-1698.9,-12645.2,2460.7,1749.1,4835.9,3845.3,1948.2,306.0,2284.4,...,53.9,187.4,1064.1,2737.4,28032.4,0.0,38882.6,8112.8,76.5,97.6
1,1672528500,-1773.9,-12544.0,2458.5,1745.9,4823.1,3847.4,1951.6,306.8,2283.1,...,39.3,187.3,1062.9,2973.8,28081.7,0.0,38564.6,7509.1,76.7,99.1
2,1672529400,-2184.5,-12526.2,2459.6,1746.0,4818.8,3859.9,1947.9,306.9,2296.9,...,39.3,187.2,1069.1,3267.6,28647.1,0.0,38438.4,6523.6,77.0,101.7
3,1672530300,-2076.4,-12515.4,2457.9,1743.0,4814.2,3861.3,1942.0,306.0,2288.7,...,39.3,187.3,1057.3,3257.6,27870.4,0.0,38260.4,7132.5,76.7,100.1
4,1672531200,-889.8,-15410.1,2457.7,1743.3,4796.5,3869.5,1933.2,306.0,2079.8,...,106.0,187.3,1060.8,3142.2,28724.5,0.0,37893.7,6027.1,77.2,102.9


# Combine and modify data

In [8]:
def get_state_abr(state):
    d = {
        'Baden-Württemberg': 'BW',
        'Bayern': 'BY',
        'Berlin': 'BE',
        'Brandenburg': 'BB',
        'Bremen': 'HB',
        'Hamburg': 'HH',
        'Hessen': 'HE',
        'Mecklenburg-Vorpommern': 'MV',
        'Niedersachsen': 'NI',
        'Nordrhein-Westfalen': 'NW',
        'Rheinland-Pfalz': 'RP',
        'Saarland': 'SL',
        'Sachsen': 'SN',
        'Sachsen-Anhalt': 'ST',
        'Schleswig-Holstein': 'SH',
        'Thüringen': 'TH'}
    return d[state]


# Import weather data
df_weather = pd.read_csv('Weather_data.csv')
df_weather['date_time'] = df_weather['time_unix'].apply(
    datetime.datetime.fromtimestamp).dt.strftime('%Y-%m-%d-%H')
df_weather = df_weather.drop(['time_unix', 'Unnamed: 0'], axis='columns')
df_weather = df_weather.set_index('date_time')


def rename(string):
    state = string.split('_')[-1]
    old_key = string.split('_')[0]+'_'+string.split('_')[1]
    new_key = old_key+'_'+get_state_abr(state)
    return new_key


df_weather = df_weather.rename(rename, axis='columns')

# Import energy data
df_energy = pd.read_csv('Energy_data.csv')
df_energy['date_time'] = df_energy['time_unix'].apply(
    datetime.datetime.fromtimestamp).dt.strftime('%Y-%m-%d-%H')
df_energy.set_index('date_time')
df_energy = df_energy.drop(['time_unix', 'Unnamed: 0'], axis='columns')
df_energy = df_energy.groupby('date_time').mean()

# Joine data frames along date_time column, create some extra features
df = df_energy.join(df_weather)
df['date_time'] = df.index
df['date_time'] = df['date_time'].apply(
    lambda string: datetime.datetime.strptime(string, '%Y-%m-%d-%H'))
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.weekday
df['date_year'] = df['date_time'].dt.strftime('%Y-%m-%d')
df['hour'] = df['date_time'].dt.strftime('%H')
df['date'] = df['date_time'].dt.strftime('%m-%d')
df = df.drop('date_time', axis='columns')

# Replace None values by median
for key in df.select_dtypes(include=np.number):
    df[key] = df[key].fillna(np.nanmedian(df[key]))

df.to_csv('data/Energy_production_weather.csv')
os.remove('Weather_data.csv')
os.remove('Energy_data.csv')